### Para entrenar librerias:

pip install: 

>python 3.10.x max

> Tensorflow 2.20

> scikit-klearn

> rasterio

> opencv-python

> segmentation_models

> albumentations

#### Creacion de train, val, test

! Las imagenes deben tener la notación correcta

## Division por idccpp




In [1]:
from utils import split_dataset_por_ccpp

(train_imgs, train_masks, val_imgs, val_masks, test_imgs, test_masks) = split_dataset_por_ccpp( carpeta_imgs="dataset_v0.9sentinel_tif/images",
                                                                                                carpeta_masks="dataset_v0.9sentinel_tif/masks",
                                                                                                test_size=0.10,
                                                                                                val_size=0.20,
                                                                                                random_state=2025
                                                                                            )



✓ División correcta sin errores de archivos.
--------------------------------------------------
TRAIN     → imágenes: 780 | máscaras: 780
VALIDATION → imágenes: 216 | máscaras: 216
TEST      → imágenes: 133 | máscaras: 133
--------------------------------------------------
TOTAL IMGS: 1129
TOTAL MASKS: 1129
--------------------------------------------------


### division por tiles

In [2]:
from TiffDataGeneratorAugmented import TIFFDataGeneratorAug

# Generadores
train_gen = TIFFDataGeneratorAug(train_imgs, train_masks, batch_size=2, n_channels=3, augment=True)#<--- canales
val_gen = TIFFDataGeneratorAug(val_imgs, val_masks, batch_size=2, shuffle=False, n_channels=3, augment=False)#<--- canales
test_gen = TIFFDataGeneratorAug(test_imgs, test_masks, batch_size=1, shuffle=False, n_channels=3, augment=False)#<--- canales


2025-11-20 09:01:24.233010: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-20 09:01:25.745440: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-20 09:01:28.914898: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
/home/rezxch/tf220/lib/python3.10/site-packages/albumentations/check_version.py:147: UserWa

In [3]:
import os
os.environ["SM_FRAMEWORK"] = "tf.keras"

from tensorflow import keras
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [4]:
from tensorflow.keras.optimizers import Adam
BACKBONE = 'resnet34'#efficientnetb0 tmb puede ser (buena con imag satelitales)
model = sm.Unet(
    backbone_name=BACKBONE,
    encoder_weights='imagenet',
    classes=1,
    activation='sigmoid'
    #input_shape=(512, 512, 4)
)

model.compile(
    optimizer=Adam(1e-4),
    loss=sm.losses.bce_jaccard_loss,
    metrics=[sm.metrics.IOUScore(), sm.metrics.FScore()]
)

I0000 00:00:1763647309.204942    1134 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 3381 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


In [5]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Early stopping: detener cuando el modelo deje de mejorar
early_stop = EarlyStopping(
    monitor='val_iou_score',      # métrica que se evalúa
    mode='max',           # porque cuanto más alto, mejor
    patience=10,
    restore_best_weights=True
)

# Guardar el mejor modelo automáticamente
checkpoint = ModelCheckpoint(
    "unet_resnet34_v0.9sentinel.keras",         # ruta del archivo
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)


In [6]:
history = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=100,
    verbose=1,
    callbacks=[early_stop,checkpoint]
)
print("Epochs realizadas:", len(history.history['loss']))

/home/rezxch/tf220/lib/python3.10/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100


2025-11-20 09:02:17.016704: I external/local_xla/xla/service/service.cc:163] XLA service 0x714be80044b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-11-20 09:02:17.016753: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-11-20 09:02:17.665809: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-20 09:02:21.453342: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91600
2025-11-20 09:02:33.673698: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.26GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-20 09:02:33.86881

390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 113ms/step - f1-score: 0.3616 - iou_score: 0.2340 - loss: 1.1040
Epoch 1: val_loss improved from None to 1.50482, saving model to unet_resnet34_v0.9sentinel.keras
390/390 ━━━━━━━━━━━━━━━━━━━━ 110s 138ms/step - f1-score: 0.4352 - iou_score: 0.2947 - loss: 0.9769 - val_f1-score: 0.0415 - val_iou_score: 0.0212 - val_loss: 1.5048
Epoch 2/100
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 114ms/step - f1-score: 0.5502 - iou_score: 0.3949 - loss: 0.8065
Epoch 2: val_loss did not improve from 1.50482
390/390 ━━━━━━━━━━━━━━━━━━━━ 48s 123ms/step - f1-score: 0.5643 - iou_score: 0.4137 - loss: 0.7841 - val_f1-score: 0.0110 - val_iou_score: 0.0055 - val_loss: 1.7252
Epoch 3/100
390/390 ━━━━━━━━━━━━━━━━━━━━ 0s 115ms/step - f1-score: 0.6143 - iou_score: 0.4646 - loss: 0.7243
Epoch 3: val_loss improved from 1.50482 to 0.82227, saving model to unet_resnet34_v0.9sentinel.keras
390/390 ━━━━━━━━━━━━━━━━━━━━ 50s 129ms/step - f1-score: 0.6279 - iou_score: 0.4800 - loss: 0.6962 - val_f1-sco

In [7]:
model.save("unet_resnet34_v0.9sentinel_model.keras")

In [8]:
model.save_weights("unet_resnet34_v0.9sentinel_model.weights.h5")

### Evaluando el modelo en train, val, test

In [9]:
train_loss, train_iou, train_f1 = model.evaluate(train_gen, verbose=1)
val_loss, val_iou, val_f1 = model.evaluate(val_gen, verbose=1)
test_loss, test_iou, test_f1 = model.evaluate(test_gen, verbose=1)


print("Resultados en TRAIN:")
print("  Loss:", train_loss)
print("  IoU :", train_iou)
print("  F1  :", train_f1)

print("Resultados en VAL:")
print("  Loss:", val_loss)
print("  IoU :", val_iou)
print("  F1  :", val_f1)

print("Resultados en TEST:")
print("  Loss:", test_loss)
print("  IoU :", test_iou)
print("  F1  :", test_f1)


390/390 ━━━━━━━━━━━━━━━━━━━━ 16s 39ms/step - f1-score: 0.7290 - iou_score: 0.6044 - loss: 0.5559
108/108 ━━━━━━━━━━━━━━━━━━━━ 3s 30ms/step - f1-score: 0.6782 - iou_score: 0.5594 - loss: 0.6268


2025-11-20 09:23:44.815531: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.23GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-20 09:23:44.964306: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.16GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2025-11-20 09:23:45.150709: W external/local_xla/xla/tsl/framework/bfc_allocator.cc:310] Allocator (GPU_0_bfc) ran out of memory trying to allocate 6.16GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


133/133 ━━━━━━━━━━━━━━━━━━━━ 9s 22ms/step - f1-score: 0.6689 - iou_score: 0.5568 - loss: 0.6141
Resultados en TRAIN:
  Loss: 0.5558902025222778
  IoU : 0.6043699383735657
  F1  : 0.7290002107620239
Resultados en VAL:
  Loss: 0.6268019080162048
  IoU : 0.5593595504760742
  F1  : 0.6781629920005798
Resultados en TEST:
  Loss: 0.6141063570976257
  IoU : 0.5567782521247864
  F1  : 0.6688633561134338
